<a href="https://colab.research.google.com/github/abhi-11nav/Text-Emotion-Detection/blob/main/Text_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# Importing the necessary libraries 

import pandas as pd
import numpy as np 

In [67]:
# Cloning the github repository 

!git clone https://github.com/abhi-11nav/Text-Emotion-Detection.git

fatal: destination path 'Text-Emotion-Detection' already exists and is not an empty directory.


In [68]:
# Importing data

data = pd.read_csv("/content/Text-Emotion-Detection/tweet_emotions.csv")

In [69]:
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [70]:
# Let us drop the tweet id

data.drop("tweet_id", axis=1, inplace=True)

In [71]:
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [72]:
# Let us check if the tweet has any missing values 

data.isna().any()

sentiment    False
content      False
dtype: bool

No missing values

In [73]:
# Let us check the number of categories in sentiment variable

data['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

The data seems imbalanced. Let us deal with it after a bit

In [74]:
# Let us look at the sentences

data['content'][0]

'@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =['

In [75]:
data['content'][1]

'Layin n bed with a headache  ughhhh...waitin on your call...'

Text Preprocessing

In [76]:
# Importing libraries

import re 

import nltk 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [77]:
def text_preprocess(dataset,list_name):
  
  for i in range(dataset.shape[0]):
    list_name.append(re.sub('[^a-zA-Z]',' ',str(dataset.iloc[i,1])))

  print("Number and other symbols eliminated from the text")

  # String spacing 
  for x in range(len(list_name)):
    list_name[x] = " ".join(y for y in str(list_name[x]).split()).lower()

  print("Text reorganized and converted to small letter")
  
  for index in range(len(list_name)):
    temp_list= []
    # Lemmatization
    for word in list_name[index].split():
      if word not in stopwords.words('english'):
        temp_list.append(word)
    
    list_name[index] = " ".join(lemmatizer.lemmatize(words) for words in temp_list )

In [78]:
sentences = []

text_preprocess(data,sentences)

Number and other symbols eliminated from the text
Text reorganized and converted to small letter


In [79]:
len(sentences)

40000

Text preprocessing done

Converting text to vectors 

Word2vec

In [80]:
# Importing necessary libraries

import gensim

from gensim.models import Word2Vec

from tqdm import tqdm

from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [81]:
# Words list

words_list = []

# looping through to append words
for index in range(len(sentences)):
  words_list.append(nltk.word_tokenize(sentences[index]))

print(len(words_list)," length of sentences")

40000  length of sentences


In [82]:
model = gensim.models.Word2Vec(words_list, window=5, min_count = 2)

In [ ]:
model.wv.index2word

In [111]:
model.wv.most_similar('joy')

[('official', 0.9989094138145447),
 ('havent', 0.9988674521446228),
 ('mine', 0.9987185001373291),
 ('including', 0.9983623623847961),
 ('making', 0.9983320236206055),
 ('everybody', 0.9983152151107788),
 ('shame', 0.9982696175575256),
 ('andrew', 0.9982351660728455),
 ('biz', 0.9980695247650146),
 ('inside', 0.9980249404907227)]

In [121]:
# Empty list 
X = []

# Looping though words
for words in tqdm(words_list):
  X.append(np.mean([model.wv[word] for word in words if word in model.wv.index2word], axis=0))

  0%|          | 0/40000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 40000/40000 [00:36<00:00, 1084.92it/s]


In [122]:
y = data["sentiment"]

In [85]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

In [153]:
X = np.array(X)
y = le.fit_transform(y)

Train test split

In [154]:
import tensorflow as tf

In [155]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.1,random_state=101)

In [156]:
train_X.shape

(36000,)

In [157]:
train_y.shape

(36000,)

In [158]:
test_X.shape

(4000,)

In [159]:
test_y.shape

(4000,)

.

## Bi-directional LSTM RNN 

Implementing Bi-directional Long short term Memory recurrent neural network 

In [160]:
# Importing the necessary libraries

import tensorflow 
from tensorflow import keras

from keras.layers import Dense, Flatten, Input, LSTM, Bidirectional, Embedding, Dropout
from keras.models import Model, Sequential

In [161]:
# Unique words

unique_word_list = []

for index in range(len(sentences)):
  [unique_word_list.append(w) for w in sentences[index].split()]


unique_words = list(set(unique_word_list))

print(len(unique_words))

42763


Sequential API

In [162]:
classes = len(data['sentiment'].unique())

print(classes)

13


In [163]:
model = Sequential()

In [164]:
# Model 

model.add(Input(shape=(100,1)))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.3))
model.add(Dense(classes, activation='softmax'))

In [167]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [165]:
model = Model(inputs = model.input, outputs=model.output)

In [166]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 1)]          0         
                                                                 
 lstm_7 (LSTM)               (None, 100, 50)           10400     
                                                                 
 dropout_6 (Dropout)         (None, 100, 50)           0         
                                                                 
 lstm_8 (LSTM)               (None, 100, 50)           20200     
                                                                 
 dropout_7 (Dropout)         (None, 100, 50)           0         
                                                                 
 dense_3 (Dense)             (None, 100, 13)           663       
                                                                 
Total params: 31,263
Trainable params: 31,263
Non-trainable

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config


In [177]:
train_X = np.expand_dims(np.array([tensorflow.convert_to_tensor(x) for x in X]),axis=-1)

ValueError: ignored

In [182]:
y = np.expand_dims(tensorflow.convert_to_tensor(y),axis=-1)

In [184]:
model.fit(tensorflow.convert_to_tensor(train_X),tensorflow.convert_to_tensor(y), epochs=2)

ValueError: ignored

In [ ]:
tra

In [ ]:
train_X.dtype

dtype('O')

In [ ]:
train_X = np.array([np.array(val).astype('float64') for val in train_X])
train_y = np.array([np.array(val).astype('float64') for val in train_y])
test_X = np.array([np.array(val).astype('float64') for val in test_X])
test_y = np.array([np.array(val).astype('float64') for val in test_y])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
model.fit(train_X, train_y, epochs=10)

ValueError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()

In [ ]:
classifier.fit(train_X, train_y)

ValueError: ignored

In [ ]:
train_X = np.array(train_X)

In [ ]:
train_y.dtype

dtype('int64')

In [ ]:
classifier.fit(train_X, train_y)

ValueError: ignored